In [ ]:
!pip install mp_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:

from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

## Extracción de los datos

Como ejemplo extraemos todos los oxidos de hierro $Fe_2O_3$ (hematita):

In [ ]:
from mp_api.client import MPRester

with MPRester("OFUWg4e1sqgL7iIZAm5an6FneEuobbI8") as mpr:
  docs = mpr.summary.search(crystal_system='Tetragonal', num_elements=2)


/usr/local/lib/python3.10/dist-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
<ipython-input-83-fcbc04924bef>:4: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(crystal_system='Tetragonal', num_elements=2)


Retrieving SummaryDoc documents:   0%|          | 0/3140 [00:00<?, ?it/s]

In [ ]:
docs = np.random.choice(docs, size=1000, replace=False)

In [ ]:
len(docs)

1000

De la estructura de cada oxido de hierro extraemos los patrones de difracción con su respectiva etiqueta (fórmula y Cristal Systema)

In [ ]:
def calcular_difractogramas(docs):
    # Inicializar el calculador de difractogramas
    calculator = XRDCalculator(wavelength="CuKa")

    # Listas para almacenar los resultados
    two_theta = []
    intensity = []
    cs = []
    formula = []

    # Agregar barra de progreso
    for doc in tqdm(docs, desc="Calculando difractogramas", unit="structure"):
        # Obtener el sistema cristalino
        struct = doc.structure
        sga = SpacegroupAnalyzer(struct)
        conventional_structure = sga.get_conventional_standard_structure()
        cs.append(sga.get_crystal_system())

        # Calcular el difractograma
        difractograma = calculator.get_pattern(conventional_structure, two_theta_range=(0, 135))
        two_theta.append(difractograma.x)
        intensity.append(difractograma.y)

        wy = str(sga.get_symmetrized_structure()).split('\n')
        formula.append(wy[2].split(' ')[-1])

    return two_theta, intensity, cs, formula

Creamos una función que nos iguale el tamaño de los difractogramas. Es decir, que los arrays de intensidades para cada difractograma tengan la misma cantidad de elementos. En este caso a partir de un del siguiente array

```Python
    rango_2theta = np.arange(0, 135, 0.01)
```

In [ ]:
def generar_perfil(posiciones, intensidades):
    rango_2theta = np.arange(0, 135, 0.01)

    # Encontrar los índices correspondientes a las posiciones de los picos
    indices_picos = np.searchsorted(rango_2theta, posiciones)

    # Asegurarse de que los índices estén dentro de los límites del array
    indices_picos = np.clip(indices_picos, 0, len(rango_2theta) - 1)

    # Crear un array con las intensidades correspondientes a cada valor de 2theta
    intensidades_2theta = np.zeros_like(rango_2theta)

    # Asignar las intensidades de los picos a los valores correspondientes de 2theta
    intensidades_2theta[indices_picos] = intensidades

    return rango_2theta, intensidades_2theta


Creamos una nueva función que aplique la función anterior a todo los difractogramas

In [ ]:
def generar_perfiles(angles, intensity):
    thetas = []
    peaks = []

    for i in tqdm(range(len(angles)), desc="Generando perfiles"):
        x, y = generar_perfil(angles[i], intensity[i])
        thetas.append(x)
        peaks.append(y)

    return thetas, peaks

In [ ]:
two_theta, intensity, cs, formula = calcular_difractogramas(docs)

Calculando difractogramas: 100%|██████████| 1000/1000 [10:56<00:00,  1.52structure/s]


In [ ]:
thetas, peaks = generar_perfiles(two_theta, intensity)

Generando perfiles: 100%|██████████| 1000/1000 [00:00<00:00, 13980.41it/s]


In [ ]:
dicc = {'intensity': peaks, 'crystal_system':cs}
df = pd.DataFrame(dicc)

In [ ]:
mask = df['crystal_system']=='tetragonal'
df = df[mask]

In [ ]:
df

,intensity,crystal_system
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
...,...,...
994,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
995,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
996,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal
998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",tetragonal


In [ ]:
def guardar_dataframe(dataframe, nombre_archivo):
    # Convertir las columnas con arrays a representación JSON antes de guardar
    dataframe_converted = dataframe.applymap(lambda x: x.tolist() if isinstance(x, list) else x)

    # Guardar el DataFrame en formato JSON
    dataframe_converted.to_json(nombre_archivo, orient='records', lines=True)
    print(f"DataFrame guardado en {nombre_archivo}")

guardar_dataframe(df, 'Tetragonal.json')

DataFrame guardado en Tetragonal.json


In [ ]:
def cargar_dataframe(nombre_archivo):
    # Leer el DataFrame desde el archivo JSON
    dataframe = pd.read_json(nombre_archivo, lines=True)

    # Convertir las listas en las columnas de nuevo a arrays
    dataframe = dataframe.applymap(lambda x: x if not isinstance(x, list) else tuple(x))

    return dataframe

df_prueba = cargar_dataframe('trigonal.json')

In [ ]:
df_prueba

,intensity,crystal_system
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
...,...,...
961,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
962,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
963,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal
964,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",trigonal


In [ ]:
dicc_classes = {
    'triclinic': 0,
    'monoclinic': 1,
    'orthorhombic': 2,
    'tetragonal': 3,
    'trigonal': 4,
    'hexagonal': 5,
    'cubic': 6
}


In [ ]:
df['Y'] = df['crystal_system'].map(dicc_classes)

In [ ]:
df

,intensity,crystal_system,formula,Y
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,Al13Si5H18ClO38,6
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,Al3Si3Ag4IO12,6
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,AlH18C3(N3F2)3,6
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,AlH18Ru(NF)6,6
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,B3H60C16(NO)12,6
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,BaCaBiSbO6,6
6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,BaCaCeCrO6,6
7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,BaCaCeFeO6,6
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,BaCaCeGeO6,6
9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",cubic,BaCaCeHfO6,6


In [ ]:
def multiclase_a_one_hot(df, columna, prefijo=''):
    one_hot = pd.get_dummies(df[columna], prefix=prefijo)
    df = pd.concat([df, one_hot], axis=1)
    df = df.drop(columna, axis=1)
    return df

In [ ]:
multiclase_a_one_hot(df, 'crystal_system', prefijo='')

,intensity,formula,Y,_cubic
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Al13Si5H18ClO38,6,1
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Al3Si3Ag4IO12,6,1
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",AlH18C3(N3F2)3,6,1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",AlH18Ru(NF)6,6,1
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",B3H60C16(NO)12,6,1
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",BaCaBiSbO6,6,1
6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",BaCaCeCrO6,6,1
7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",BaCaCeFeO6,6,1
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",BaCaCeGeO6,6,1
9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",BaCaCeHfO6,6,1
